In [0]:
from pyspark.sql import functions as F

src_path = "s3://paysim-bronze-eng/ps_20174392719_1491204439457_log.csv"
df = (spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(src_path))
print(f"Rows read: {df.count()}")
display(df.limit(10))



In [0]:
%sql
CREATE CATALOG IF NOT EXISTS keystone
MANAGED LOCATION 's3://paysim-bronze-eng/';

CREATE SCHEMA IF NOT EXISTS keystone.silver;
CREATE SCHEMA IF NOT EXISTS keystone.gold;
